## Imports

In [ ]:
import pandas as pd
import numpy as np
import time

from sklearn import svm
from sklearn import metrics

import joblib

## Globals

In [ ]:
CLASS_LABELS = ['non-violent','violent']
LABELS_COUNT = len(CLASS_LABELS)

DATASET_PATH = 'Dataset/violent_keypoints_dataset.csv'
CONF_DATASET_PATH = 'Dataset/violent_keypoints_conf_dataset.csv'

MEAN_CONF_THRESHOLD = 0.3

TRAIN_SPLIT = 0.8

EPOCHS = 50
BATCH_SIZE = 32

## Import Dataset

In [ ]:
#Import Dataset
data = pd.read_csv(DATASET_PATH).values

data[data == "non-violent"] = 0
data[data == "violent"] = 1

data = data.astype('float32')

print(data)
print(data.shape)

## Import Confidence Scores

In [ ]:
#Import Confidence
conf_data = pd.read_csv(CONF_DATASET_PATH).values

conf_data = conf_data[:, 1:]

conf_data = conf_data.astype('float32')

print(conf_data)
print(conf_data.shape)

## Remove Data w/ Low Confidence

In [ ]:
high_conf_data = []

count = 0

#Mean Confidence
for x in range(len(conf_data)):
    count += np.mean(conf_data[x])
    if np.mean(conf_data[x]) >= MEAN_CONF_THRESHOLD:
        high_conf_data.append(data[x])

avg_mean = count / len(conf_data)
print("AVG MEAN: " + str(avg_mean))

high_conf_data = np.array(high_conf_data)

print(high_conf_data)
print(high_conf_data.shape)

## Split Dataset

In [ ]:
TRAIN_AMOUNT = int(data.shape[0] * TRAIN_SPLIT)

#Shuffle Data
for x in range(100):
    np.random.shuffle(data)

#Split Data into Train and Test
train = data[0:TRAIN_AMOUNT]
test = data[TRAIN_AMOUNT:]

#Extract Data and Reshape to 2d array
x_train = np.reshape(train[:, 1:],(train.shape[0],19,3,2)).swapaxes(2,3).swapaxes(1,3)
x_train = np.reshape(x_train,(train.shape[0],3,38), order="F")
x_train = np.reshape(x_train,(x_train.shape[0],3 * 38))

x_test = np.reshape(test[:, 1:],(test.shape[0],19,3,2)).swapaxes(2,3).swapaxes(1,3)
x_test = np.reshape(x_test,(test.shape[0],3,38), order="F")
x_test = np.reshape(x_test,(x_test.shape[0],3 * 38))

#Extract Labels
y_train = train[:,0]
y_test = test[:,0]

## Build SVM Model

In [ ]:
#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel

## Train Model

In [ ]:
#Start taking time
start = time.time()

#Train the model using the training sets
clf.fit(x_train, y_train)

#Output time taken
print(f'Time: {time.time() - start}')

#Predict the response for test dataset
y_pred = clf.predict(x_test)

## Model Evaluation

In [ ]:
# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(y_test, y_pred))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, y_pred))


## Export Model and Weights

In [ ]:
#Export SVM model
joblib.dump(clf, 'ViolentModel/SVM/violent_model.pkl')
